In [6]:
# import dependencies 
from fastai.vision.all import *
import timm
from torchvision import transforms
from pathlib import Path
from utils import get_y, create_dls, get_mean_table, dataset_forecast, adapt_model_to_new_dls, create_dls_stratified

In [8]:
# Usage
dls = create_dls_stratified(train_pct=0.8, path='data/FieldPlant/')

TypeError: create_dls_stratified() got an unexpected keyword argument 'train_pct'

In [4]:

model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
learn = Learner(dls, model, metrics=accuracy)
learn.fine_tune(epochs=1)

TypeError: expected str, bytes or os.PathLike object, not float

In [ ]:
def adapt_model_to_new_dls(learn, dls):
    # Update the learner's dataloaders
    learn.dls = dls
    # Get the number of classes in the new dataset
    num_classes_new = dls.c
    # Replace the final layer of the model to match the new number of classes
    # Assuming the final layer is named 'classifier' which is common in timm models
    # Note: The naming might vary, so ensure to adjust according to your model's architecture
    learn.model.classifier = nn.Linear(learn.model.classifier.in_features, num_classes_new)
    # Now you can evaluate the model on the new dataset
    return learn

In [ ]:
learn2 = adapt_model_to_new_dls(learn, dls)

In [ ]:
learn2.fine_tune(1)

In [ ]:
model.num_classes

In [ ]:
model.num_classes = 2

In [ ]:
learn.fine_tune(epochs=1)

In [ ]:
def iteratively_train_on_dataset_fractions(train_pcts = [0.1, 0.25, 0.5, 0.65], replicates=1, dataset_path):
    results_table = pd.DataFrame(columns=['train_pct', 'acc'])
    for train_pct in train_pcts:
        print(f'Creating DL: {train_pct}')
        dls = create_dls(train_pct, dataset_path)
        for replicate in range(replicates): # repeat training and average results for increased accuracy 
            print(f'{train_pct} replicate {replicate}')
            # fine tune model
            model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
            learn = Learner(dls, model, metrics=accuracy)
            learn.fine_tune(epochs=15)
            val_acc = learn.recorder.values[-1][2]
            row = pd.DataFrame({'train_pct': train_pct, 'acc': val_acc}, index=[0])
            results_table = pd.concat([results_table, row])
    mean_table = get_mean_table(results_table)
    return mean_table

In [ ]:
train_pcts = [0.1, 0.25, 0.5, 0.65]
replicates = 3
results_table = pd.DataFrame(columns=['train_pct', 'acc'])
for train_pct in train_pcts:
    print(f'Creating DL: {train_pct}')
    dls = create_dls(train_pct)
    for replicate in range(replicates): # repeat training and average results for increased accuracy 
        print(f'{train_pct} replicate {replicate}')
        # fine tune model
        model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
        learn = Learner(dls, model, metrics=accuracy)
        learn.fine_tune(epochs=15)
        val_acc = learn.recorder.values[-1][2]
        row = pd.DataFrame({'train_pct': train_pct, 'acc': val_acc}, index=[0])
        results_table = pd.concat([results_table, row])

mean_table = get_mean_table(results_table)

In [ ]:
polynomial = dataset_forecast(mean_table)

In [ ]:
percentage_of_data = 0.9
predicted_accuracy = polynomial(percentage_of_data)
predicted_accuracy

In [ ]:
train_pcts = [0.9]
replicates = 3
results_table = pd.DataFrame(columns=['train_pct', 'acc'])
for train_pct in train_pcts:
    print(f'Creating DL: {train_pct}')
    dls = create_dls(train_pct)
    for replicate in range(replicates): # repeat training and average results for increased accuracy 
        print(f'{train_pct} replicate {replicate}')
        # fine tune model
        model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
        learn = Learner(dls, model, metrics=accuracy)
        learn.fine_tune(epochs=15)
        val_acc = learn.recorder.values[-1][2]
        row = pd.DataFrame({'train_pct': train_pct, 'acc': val_acc}, index=[0])
        results_table = pd.concat([results_table, row])

mean_table = pd.concat([mean_table, get_mean_table(results_table)])
mean_table = mean_table.reset_index()

In [ ]:
# plot polynomial here
polynomial = dataset_forecast(mean_table)

In [ ]:
percentage_of_data = 0.95
predicted_accuracy = polynomial(percentage_of_data)
predicted_accuracy

In [ ]:
# Resnet-18
# epoch	train_loss	valid_loss	accuracy	time
# 0	2.329669	2.175517	0.398039	04:44
# 1	2.047739	1.710715	0.527451	04:47
# 2	1.696105	1.822478	0.431373	04:57
# 3	1.373082	1.125379	0.621569	05:14
# 4	1.118939	1.178962	0.613725	05:16
# 5	0.903016	1.063396	0.656863	05:16
# 6	0.726309	0.970530	0.662745	05:16
# 7	0.587566	0.937186	0.670588	05:15
# 8	0.492905	0.925825	0.688235	05:04
# 9	0.435941	0.922592	0.692157	04:56

In [ ]:
learn.save('fine_tuned_model')

In [ ]:
train_pcts = [0.1, 0.25, 0.5, 0.65]
replicates = 3
field_plant_results_table = pd.DataFrame(columns=['train_pct', 'acc'])
for train_pct in train_pcts:
    print(f'Creating DL: {train_pct}')
    dls = create_dls(train_pct, 'plantvillage-dataset/color')
    for replicate in range(replicates): # repeat training and average results for increased accuracy 
        print(f'{train_pct} replicate {replicate}')
        # fine tune model
        model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
        learn = Learner(dls, model, metrics=accuracy)
        learn.fine_tune(epochs=15)
        val_acc = learn.recorder.values[-1][2]
        row = pd.DataFrame({'train_pct': train_pct, 'acc': val_acc}, index=[0])
        field_plant_results_table = pd.concat([field_plant_results_table, row])

In [ ]:
# polynomial = dataset_forecast(field_plant_results_table)

In [ ]:
train_pcts = [0.1, 0.25, 0.5, 0.65]
replicates = 3
field_plant_results_table = pd.DataFrame(columns=['train_pct', 'acc'])
for train_pct in train_pcts:
    print(f'Creating DL: {train_pct}')
    dls = create_dls(train_pct, 'plantvillage-dataset/segmented')
    for replicate in range(replicates): # repeat training and average results for increased accuracy 
        print(f'{train_pct} replicate {replicate}')
        # fine tune model
        model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
        learn = Learner(dls, model, metrics=accuracy)
        learn.fine_tune(epochs=15)
        val_acc = learn.recorder.values[-1][2]
        row = pd.DataFrame({'train_pct': train_pct, 'acc': val_acc}, index=[0])
        field_plant_results_table = pd.concat([field_plant_results_table, row])

In [ ]:
dls = create_dls(0.8, 'plantvillage-dataset/segmented')
# for replicate in range(replicates): # repeat training and average results for increased accuracy 
# print(f'{train_pct} replicate {replicate}')
# # fine tune model
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
learn = Learner(dls, model, metrics=accuracy)
learn.fine_tune(epochs=15)

In [ ]:
dls = create_dls(0.8, 'plantvillage-dataset/color')
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=dls.c)
learn = Learner(dls, model, metrics=accuracy)
learn.fine_tune(epochs=1)
learn.recorder.plot_loss() 

In [ ]:
# Load the new dataset
dls = create_dls_stratified(0.5, 'FieldPlant-dataset')

In [ ]:
learn2 = adapt_model_to_new_dls(learn, dls)    
learn2.validate()
